In [1]:
import os
import sys

# Step 2.1: Set the path to your Django app directory
sys.path.append(os.path.abspath(os.path.join('..', '..', 'backend', 'kit_hunter')))  # Adjust path as necessary

# Step 2.2: Set up Django environment
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'kit_hunter.settings')  # Adjust according to your settings module

import django
django.setup()  # Initialize Django

In [3]:
import threading

from app.models import League, Team
from django.db.models import Count  # Import Count

def get_leagues():
    leagues = League.objects.all()
    return leagues

def main():

    leagues = get_leagues()

    for league in leagues:
        teams = Team.objects.filter(league_id__exact=league.id).values('country').annotate(count=Count('id')).order_by('-count')
        print(len(teams))
        if teams:
            league.country = teams[0]["country"]
            league.save()


# Running in a separate thread
thread = threading.Thread(target=main)
thread.start()
thread.join()  # Wait for the thread to finish

2
1
2
1
1
0
1
1
1
2
1
1
1
0
0
1
2
1
1
1
1
1
1
1
0
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
2
1
1
2
1
1
1
0
1
2
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
1
0
0
1
1
0
0
0
0
1
1
1
2
1
1
2
0
0
1
1
1
1
0
0
0
1
0
0
1
1
0
0
1
1
1
1
1
0
4
0
1
0
1
3
1
1
0
1
1
0
1
0
0
0
1
0
1
1
0
1
0
1
1
1
